In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from random import choice
pd.options.mode.chained_assignment = None 
import pickle

In [6]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Coding/Python'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
# from lonelyboy.geospatial import group_patterns as gsgp


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import PyQt5
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')

style.use('ggplot')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

In [7]:
PLT_FIG_WIDTH = 3.787
PLT_FIG_HEIGHT = PLT_FIG_WIDTH / 1.618

# Reading Data (Testing on Fisheries)

In [8]:
ports = pd.read_pickle('./data/pkl/ports_raw.pkl')
# fisheries = pd.read_csv('data/csv/fisheries_dynamic_real_points_v2.csv')

# fisheries = gspp.gdf_from_df(fisheries, crs={'init':'epsg:4326'})
ports = gspp.create_port_bounds(ports, epsg=2154, port_radius=2000)

In [ ]:
gsplt.map_plot(fisheries, color=['steelblue'], title='Testing Trajectories', fontsize=10)

# [TESTING] Calculating AIS Activity 

In [7]:
fisheries.drop(['index'], axis=1, inplace=True)

In [8]:
fisheries.head()

,id,mmsi,turn,speed,course,heading,lon,lat,ts,velocity,geom,traj_id,trip_id
0,13729318,227443000,-127.0,11.1,171.5,511,-4.782632,48.005634,1443650411,12.628942,POINT (-4.782632 48.005634),0.0,0
1,13729321,227443000,-127.0,11.3,172.6,511,-4.782532,48.005116,1443650420,10.235945,POINT (-4.7825317 48.00511599999999),0.0,0
2,13729323,227443000,-127.0,11.3,173.7,511,-4.782448,48.004600,1443650431,12.154994,POINT (-4.7824483 48.0046),0.0,0
3,13729325,227443000,-127.0,11.5,171.7,511,-4.782232,48.003550,1443650450,10.626990,POINT (-4.782232 48.00355),0.0,0
4,13729328,227443000,-127.0,11.4,171.1,511,-4.782132,48.003067,1443650460,12.686616,POINT (-4.7821317 48.003067),0.0,0


* ## Outside Port

In [61]:
ais_activity_outside_port = fisheries.groupby(['mmsi', pd.to_datetime(fisheries.ts, unit='s').dt.date, 'trip_id'], group_keys=False)\
                                        .apply(lambda df: df.ts.diff().sum()/3600).to_frame().reset_index()
ais_activity_outside_port.columns = ['mmsi', 'date', 'trip_id', '#hrs']

ais_activity_outside_port = ais_activity_outside_port.groupby(['mmsi', 'date'], group_keys=False).apply(lambda df: np.around(df['#hrs'].sum(), 3)).to_frame().reset_index()
ais_activity_outside_port.columns = ['mmsi', 'date', '#hrs']

ais_activity_outside_port

,mmsi,#hrs
0,219017843,4.369
1,224038000,68.248
2,224040000,19.177
3,224045260,13.166
4,224052970,56.448
5,224067130,81.161
6,224113270,37.843
7,224130870,85.678
8,224141980,16.803
9,224196160,11.569


* ## Within Port

In [74]:
ais_activity_within_port = pd.DataFrame([], columns=['mmsi', '#hrs'])

for (vesselId), df in fisheries.groupby(['mmsi']):
    breaking_points = df.loc[df.traj_id.diff() == 1].index.tolist()
    
    if (len(breaking_points) == 0):
        ais_activity_within_port = ais_activity_within_port.append(pd.DataFrame([[vesselId, 0]], columns=['mmsi', '#hrs']))
        continue
    
    hrs = np.around(np.sum([df.loc[bp-1:bp,:].ts.diff().values[1] for bp in breaking_points])/3600, 3)
    ais_activity_within_port = ais_activity_within_port.append(pd.DataFrame([[vesselId, hrs]], columns=['mmsi', '#hrs']))
    
ais_activity_within_port.reset_index(drop=True, inplace=True)
ais_activity_within_port

,mmsi,#hrs
0,219017843,2676.874
1,224038000,0.000
2,224040000,0.000
3,224045260,0.000
4,224052970,8.172
5,224067130,0.000
6,224113270,0.000
7,224130870,0.000
8,224141980,0.000
9,224196160,0.000


* ## Temporal Gap

In [85]:
ais_activity_temporal_gap = pd.DataFrame([], columns=['mmsi', 'traj_id', '#hrs'])

for (vesselId, trajId), df in fisheries.groupby(['mmsi', 'traj_id']):
    breaking_points = df.loc[df.trip_id.diff() == 1].index.tolist()
    
    if (len(breaking_points) == 0):
        ais_activity_temporal_gap = ais_activity_temporal_gap.append(pd.DataFrame([[vesselId, trajId, 0]], columns=['mmsi', 'traj_id', '#hrs']))
        continue
    
    hrs = np.around(np.sum([df.loc[bp-1:bp,:].ts.diff().sum() for bp in breaking_points])/3600, 3)
    ais_activity_temporal_gap = ais_activity_temporal_gap.append(pd.DataFrame([[vesselId, trajId, hrs]], columns=['mmsi', 'traj_id', '#hrs']))

ais_activity_temporal_gap.reset_index(drop=True, inplace=True)
ais_activity_temporal_gap = ais_activity_temporal_gap.groupby(['mmsi'], group_keys=False).apply(lambda df: np.around(df['#hrs'].sum(), 3)).to_frame().reset_index()
ais_activity_temporal_gap.columns = ['mmsi', '#hrs']
ais_activity_temporal_gap

,mmsi,#hrs
0,219017843,0.000
1,224038000,677.592
2,224040000,0.000
3,224045260,12.165
4,224052970,1687.470
5,224067130,2667.930
6,224113270,2912.087
7,224130870,3042.163
8,224141980,0.000
9,224196160,1396.588


# Reading and Joining CSVs

In [5]:
data_folder = './data/csv/stats/nari_dynamic_ais_activity_report_V2/'

nari_dynamic_ais_activity_report_outside_port = []
nari_dynamic_ais_activity_report_within_port = []
nari_dynamic_ais_activity_report_temporal_gaps = []

for cnt, file in enumerate(os.listdir(data_folder)):
    tmp = pd.read_csv(os.path.join(data_folder, file))
    if 'outside' in file: 
        print (f'\'outside\' in: {file}')
        nari_dynamic_ais_activity_report_outside_port.append(tmp)
    elif 'within' in file: 
        print (f'\'within\' in: {file}')
        nari_dynamic_ais_activity_report_within_port.append(tmp)
    elif 'temporal' in file: 
        print (f'\'temporal\' in: {file}')
        nari_dynamic_ais_activity_report_temporal_gaps.append(tmp)
        
nari_dynamic_ais_activity_report_outside_port = pd.concat(nari_dynamic_ais_activity_report_outside_port, ignore_index=True)
nari_dynamic_ais_activity_report_within_port = pd.concat(nari_dynamic_ais_activity_report_within_port, ignore_index=True)
nari_dynamic_ais_activity_report_temporal_gaps = pd.concat(nari_dynamic_ais_activity_report_temporal_gaps, ignore_index=True)

'outside' in: nari_dynamic_ais_activity_report_outside_port_0_Without_Dates.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_1_Without_Dates.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_2_Without_Dates.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_3_Without_Dates.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_4_Without_Dates.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_5_Without_Dates.csv
'temporal' in: nari_dynamic_ais_activity_report_temporal_gaps_0_Without_Dates.csv
'temporal' in: nari_dynamic_ais_activity_report_temporal_gaps_1_Without_Dates.csv
'temporal' in: nari_dynamic_ais_activity_report_temporal_gaps_2_Without_Dates.csv
'temporal' in: nari_dynamic_ais_activity_report_temporal_gaps_3_Without_Dates.csv
'temporal' in: nari_dynamic_ais_activity_report_temporal_gaps_4_Without_Dates.csv
'temporal' in: nari_dynamic_ais_activity_report_temporal_gaps_5_Without_Dates.csv
'within' in: nari_dynamic_ai

In [7]:
nari_dynamic_ais_activity_report_outside_port.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_outside_port_v2.csv', index=False, header=True)
nari_dynamic_ais_activity_report_within_port.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_within_port_v2.csv', index=False, header=True)
nari_dynamic_ais_activity_report_temporal_gaps.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_temporal_gaps_v2.csv', index=False, header=True)

# Making the Pie Chart

In [37]:
df = pd.DataFrame({'#Hours': [nari_dynamic_ais_activity_report_outside_port['#hrs'].sum(), nari_dynamic_ais_activity_report_within_port['#hrs'].sum(), nari_dynamic_ais_activity_report_temporal_gaps['#hrs'].sum()]},
                 index=[r'\textbf{Activity outside port}', r'\textbf{Activity inside port}', r'\textbf{Unknown (or in-) activity}'])
print (df)
explode = (0.1, 0.1, 0.1)  
ax = df.plot.pie(y='#Hours', 
#                    figsize=(PLT_FIG_WIDTH, PLT_IMAGE_HEIGHT),
                   figsize=(PLT_FIG_WIDTH, 3.7),
                   explode=explode,
                   startangle=55,
                   autopct='%1.1f\%%', 
                   pctdistance=0.80,
                   cmap='tab20c',
                   legend=False)
ax.axis('equal')

for text in ax.texts:
    text.set_fontsize(8)
    if ' ' in text.get_text():
        text.set_color('grey')
    if '%' in text.get_text():
        text.set_color('black')
        text.set_weight('bold')

plt.ylabel('')

plt.savefig('../AIS_Activity_Pie_Chart_V4.pdf', dpi=300, bbox_inches='tight')

                                         #Hours
\textbf{Activity outside port}       120968.342
\textbf{Activity inside port}        624431.903
\textbf{Unknown (or in-) activity}  2523237.096


In [26]:
PLT_FIG_HEIGHT

2.3405438813349813

# Making the Distributions

In [7]:
vessel_AIS_activity = pd.concat([nari_dynamic_ais_activity_report_outside_port, nari_dynamic_ais_activity_report_within_port, nari_dynamic_ais_activity_report_temporal_gaps], 
          axis=1, join='inner', ignore_index=True)

vessel_AIS_activity.drop([2,4], axis=1, inplace=True)
vessel_AIS_activity.columns = ['mmsi', '#hrs Outside', '#hrs Within', '#hrs Temp. Gaps']
vessel_AIS_activity

,mmsi,#hrs Outside,#hrs Within,#hrs Temp. Gaps
0,227088590,1.446,0.000,0.000
1,227318020,310.350,2725.270,80.371
2,227369960,17.147,3345.765,0.000
3,227430000,84.628,1949.508,2292.093
4,227806500,222.154,2441.440,430.760
5,228017700,1187.355,3191.717,0.000
6,228037600,433.552,3689.249,269.166
7,228037900,293.665,592.161,1484.834
8,228167900,379.371,3525.659,485.427
9,228213700,206.849,4183.748,0.000


# Basic AIS Statistics 

In [48]:
print (f' #Vessels: {len(vessel_AIS_activity.mmsi.unique())}')

 #Vessels: 3545


In [49]:
print ('Average #hrs Outside Port:\t', np.around(vessel_AIS_activity['#hrs Outside'].mean()))
print ('Cumulative #hrs Outside Port:\t', np.around(vessel_AIS_activity['#hrs Outside'].sum()))

Average #hrs Outside Port:	 34.0
Cumulative #hrs Outside Port:	 120968.0


In [50]:
print ('Average #hrs Within Port:\t', np.around(vessel_AIS_activity['#hrs Within'].mean()))
print ('Cumulative #hrs Within Port:\t', np.around(vessel_AIS_activity['#hrs Within'].sum()))

Average #hrs Within Port:	 176.0
Cumulative #hrs Within Port:	 624432.0


In [51]:
print ('Average #hrs on Temp. Gaps:\t', np.around(vessel_AIS_activity['#hrs Temp. Gaps'].mean()))
print ('Cumulative #hrs on Temp. Gaps:\t', np.around(vessel_AIS_activity['#hrs Temp. Gaps'].sum()))

Average #hrs on Temp. Gaps:	 712.0
Cumulative #hrs on Temp. Gaps:	 2523237.0


# AIS Distributions

In [44]:
out = pd.cut(vessel_AIS_activity['#hrs Outside'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 3912])
# out = pd.cut(vessel_AIS_activity['#hrs Within'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 3912]) 

ax = out.value_counts(sort=False).plot.bar(figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT), fontsize=8, width=0.75, cmap='tab20', rot=40)
# ax = out.value_counts(sort=False).plot.bar(ax=ax, figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT), fontsize=8, width=0.75, cmap='tab20', rot=40)

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.suptitle(r'\textbf{Vessel AIS Activity Outside Port}', fontsize=8, y=1)
plt.xlabel(r'\textbf{\#hrs}', fontsize=8)
plt.ylabel(r'\textbf{\#Vessels}', fontsize=8)

plt.savefig(os.path.join('..', 'AIS_Activity_Outside_Port.pdf'), dpi=300, bbox_inches='tight')

In [9]:
out = pd.cut(vessel_AIS_activity['#hrs Within'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 4325]) 
out2 = pd.cut(vessel_AIS_activity['#hrs Temp. Gaps'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 3912, 4096, 4325]) 

df1 = out.value_counts(sort=False).to_frame()
df2 = out2.value_counts(sort=False).to_frame()
df = pd.concat([df1, df2], axis=1, join='inner', ignore_index=True)
# df.columns = ['\#Hrs Within Port', '\#Hrs Temp. Gaps']
df.columns = ['\#Hrs Within Port', '\#Hrs Unknown (or in-) Activity']


x = np.arange(len(df))
w = 0.4
cmap = mpl.cm.get_cmap('tab20')

fig, ax1 = plt.subplots(figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT))
#plt.xticks(), will label the bars on x axis with the respective country names.
plt.xticks(x + w /2, df.index.values, rotation=40)
ax2 = ax1.twinx()

outside_port = ax1.bar(x, df.iloc[:,0], width=w, color=cmap(0), align='center')
temporal_gaps = ax2.bar(x+w, df.iloc[:,1], width=w, color=cmap(19), align='center')

plt.title(r'\textbf{Vessel AIS Activity Within Port \& Unknown (or in-) Activity}', fontsize=8, y=1.1)
ax1.set_xlabel(r'\textbf{\#hrs}', fontsize=8)
ax1.set_ylabel(r'\textbf{\#Vessels}', fontsize=8)
# ax2.set_ylabel(r'\textbf{\#Vessels}', fontsize=8)

plt.legend([outside_port, temporal_gaps], ['\#Hrs Within Port', '\#Hrs Unknown (or in-) Activity'], fontsize='small', loc='upper center',
          bbox_to_anchor=(0.535, 1.14), fancybox=False, shadow=False, frameon=False, ncol=2)

plt.show()
plt.savefig(os.path.join('..', 'AIS_Activity_Within_Port_Temp_Gaps_V4.pdf'), dpi=300, bbox_inches='tight')

In [10]:
out = pd.cut(vessel_AIS_activity['#hrs Within'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 4325]) 

ax = out.value_counts(sort=False).plot.bar(figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT), fontsize=8, width=0.75, cmap='tab20', rot=40)
# ax = out.value_counts(sort=False).plot.bar(ax=ax, figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT), fontsize=8, width=0.75, cmap='tab20', rot=40)

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.suptitle(r'\textbf{Vessel AIS Activity Inside Port}', fontsize=8, y=1)
plt.xlabel(r'\textbf{\#hrs}', fontsize=8)
plt.ylabel(r'\textbf{\#Vessels}', fontsize=8)

plt.savefig(os.path.join('..', 'AIS_Activity_Within_Port_V5.pdf'), dpi=300, bbox_inches='tight')

# OLD PLOTS

In [47]:
out = pd.cut(vessel_AIS_activity['#hrs Within'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 4325]) 

ax = out.value_counts(sort=False).plot.bar(figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT), fontsize=8, width=0.75, cmap='tab20', rot=40)

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.suptitle(r'\textbf{Cumulative AIS Activity Within Port}', fontsize=8, y=1)
plt.xlabel(r'\textbf{\#hrs}', fontsize=8)
plt.ylabel(r'\textbf{\#Vessels}', fontsize=8)

Text(0, 0.5, '\\textbf{\\#Vessels}')

In [49]:
out = pd.cut(vessel_AIS_activity['#hrs Temp. Gaps'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 3912, 4096, 4325]) 

ax = out.value_counts(sort=False).plot.bar(figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT), fontsize=8, width=0.75, cmap='tab20', rot=40)
# ax.set_xticklabels([str(c).split(' ')[1].split(']')[0] for c in out.cat.categories])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.suptitle(r'\textbf{Vessel AIS Activity Within Port}', fontsize=8, y=1)
plt.xlabel(r'\textbf{\#hrs}', fontsize=8)
plt.ylabel(r'\textbf{\#Vessels}', fontsize=8)

Text(0, 0.5, '\\textbf{\\#Vessels}')

In [22]:
out = pd.cut(vessel_AIS_activity['#hrs Within'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 4325]) 
out2 = pd.cut(vessel_AIS_activity['#hrs Temp. Gaps'], [0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 3912, 4096, 4325]) 

df1 = out.value_counts(sort=False).to_frame()
df2 = out2.value_counts(sort=False).to_frame()
df = pd.concat([df1, df2], axis=1, join='inner', ignore_index=True)
df.columns = ['\#Hrs Within Port', '\#Hrs Temp. Gaps']

fig, ax = plt.subplots()
ax2 = df.plot(figsize=(PLT_FIG_WIDTH,PLT_FIG_HEIGHT), kind='bar', secondary_y='\#Hrs Temp. Gaps', 
             cmap='tab20', rot=40, width=0.75, legend=False, ax=ax)   

plt.title(r'\textbf{Vessel AIS Activity Outside Port \& Temporal Gaps}', fontsize=8, y=1.1)
plt.xlabel(r'\textbf{\#hrs}', fontsize=8)
plt.ylabel(r'\textbf{\#Vessels}', fontsize=8)

ax.set_ylabel(r'\textbf{\#Vessels}', fontsize=8)

ax.legend(['\#Hrs Within Port', '\#Hrs Temp. Gaps'], fontsize='small', loc='upper center',
          bbox_to_anchor=(0.535, 1.14), fancybox=False, shadow=False, frameon=False, ncol=2)
plt.grid(True)
plt.show()

plt.savefig(os.path.join('..', 'AIS_Activity_Outside_Port_Temp_Gaps_V2.pdf'), dpi=300, bbox_inches='tight')